In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_Review_Analysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [9]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   25933450| RJOVP071AVAJO|0439873800|      84656342|There Was an Old ...|           Books|          5|            0|          0|   N|                Y|          Five Stars|I love it and so ...| 2015-08-31|
|         US|    1801372|R1ORGBETCDW3AI|1623953553|     729938122|      I Saw a Friend|           Books|          5|    

In [12]:
df.count()

10319090

In [13]:
df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'string'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

In [14]:
total_df=df.filter(df['total_votes']>=20)
total_df.show()

In [15]:
total_df.count()

161595

In [20]:
total_helpful_df = total_df.filter((total_df["helpful_votes"]/total_df["total_votes"])>=0.5)
total_helpful_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51227971|R3T6LR4H11XRAN|1585422959|     173554198|Parenting from th...|           Books|          5|           26|         26|   N|                Y|A MUST READ FOR A...|The reviewers tha...| 2015-08-31|
|         US|   53083708|R3QFNFMFWOQIBO|1619020653|     277126951|Jesus Land: A Memoir|           Books|          3|    

In [21]:
total_helpful_df.count()

114309

In [24]:
total_helpful_Y_df = total_df.filter(total_df["vine"]=="Y")

In [25]:
total_helpful_Y_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   53052945|R3KKWNG5VABDYC|1594631743|     410991634|Lafayette in the ...|           Books|          5|           31|         32|   Y|                N|Bringing America'...|Sarah Vowell has ...| 2015-08-31|
|         US|   38666248|R397PIJYYVJ7PD|0385537565|     114223774|Empire of Self: A...|           Books|          4|    

In [27]:
total_helpful_N_df = total_df.filter(total_df["vine"]=="N")
total_helpful_N_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51227971|R3T6LR4H11XRAN|1585422959|     173554198|Parenting from th...|           Books|          5|           26|         26|   N|                Y|A MUST READ FOR A...|The reviewers tha...| 2015-08-31|
|         US|   53083708|R3QFNFMFWOQIBO|1619020653|     277126951|Jesus Land: A Memoir|           Books|          3|    

In [29]:
# Determine the percentage of five-star reviews among Vine reviews where Vine=Y
vine_number = total_helpful_Y_df.count()
vine_five_star = total_helpful_Y_df.filter(total_helpful_Y_df['star_rating']== 5).count()
percentage_five_star = float(vine_five_star)*100 / float(vine_number)
print(f"The total number of reviews is {vine_number}, the number of 5-star reviews is {vine_five_star}, and the percentage of 5-star reviews vine number is {percentage_five_star}%.")

The total number of reviews is 5333, the number of 5-star reviews is 2069, and the percentage of 5-star reviews vine_number is 38.79617476092256%.


In [31]:
# Determine the percentage of five-star reviews among Vine reviews where Vine=N
vine_number_N = total_helpful_N_df.count()
vine_five_star_N = total_helpful_N_df.filter(total_helpful_N_df['star_rating']== 5).count()
percentage_five_star_N = float(vine_five_star_N)*100 / float(vine_number_N)
print(f"The total number of reviews is {vine_number_N}, the number of 5-star reviews is {vine_five_star_N}, and the percentage of 5-star reviews vine number is {percentage_five_star_N}%.")

The total number of reviews is 156262, the number of 5-star reviews is 56733, and the percentage of 5-star reviews vine_number is 36.306331673727456%.
